Rigid transformations, coordinates and reference frames
=======================================================

The [geometry](api/kineticstoolkit.geometry.rst) module simplifies the calculation of linear algebric operations on
series of points, vectors and transformation matrices. Contrarily to most other modules, this module does not work
with TimeSeries directly, but instead with the TimeSeries data. The convention for dimension order is strict but
well defined.

Dimension convention
--------------------

In this module, every point, vector or matrix is considered as a **series**, even constants which are series of 1. The dimensions of these series are:

- Dimension 0 = time (length of m). Constants are expressed as series with m = 1 and are broadcasted to match other series with m > 1.

- Dimension 1 (optional) = axis or row (length of 4).

- Dimension 2 (optional) = point/vector index or column (length of n).

To understand better how to express series of scalars, vectors, points, sets and matrices, please refer to the following pictures:

### Series of scalars ###

![Series of m scalars](_static/geometry/series_of_m_scalars.png){ width=400px }

### Series of vectors and points ###

![Series of m vectors](_static/geometry/series_of_m_vectors.png){ width=400px } ![Series of m points](_static/geometry/series_of_m_points.png){ width=400px }

Note the difference between vectors and points: vectors (e.g., forces, velocities) have zeros as their fourth coordinate while points (e.g., marker coordinates) have ones. This difference has an impact on algeabric calculation:

- A rigid transformation of a vector gives a new rotated vector expressed as a new orientation and amplitude.
- A rigid transformation on a point yields a translated point expressed as its new coordinates in the new reference frame.

### Series of groups of vectors or points ###

![Series of m groups of n vectors](_static/geometry/series_of_m_groups_of_n_vectors.png){ width=400px } ![Series of m groups of n points](_static/geometry/series_of_m_groups_of_n_points.png){ width=400px }

### Series of transformation matrices ###

![Series of m transformation matrices](_static/geometry/series_of_m_transformation_matrices.png){ width=500px }

### Working with constants ###

Since everything in the `geometry` module is considered as a series, always ensure that the first dimension of any array is reserved to time. For example, the vector (x = 1, y = 2, z = 3) must be expressed as `np.array([[1.0, 2.0, 3.0, 0.0]])` (note the double brackets). A common error would be to express it as `np.array([1.0, 2.0, 3.0, 0.0])` (single brackets), which would mean a series of 4 floats instead of one constant vector.

A quick way to convert a constant array to a series is to use numpy's `newaxis`:

In [ ]:
import kineticstoolkit.lab as ktk
import numpy as np

one_matrix = np.eye(4)
one_matrix

In [ ]:
series_of_one_matrix = one_matrix[np.newaxis]
series_of_one_matrix

Matrix multiplication
-------------------------

To facilitate the multiplication of series, kineticstoolkit.geometry provides a `matmul` function that matches both matrices' time dimensions before applying the numpy's @ or * operator accordingly on each time iteration. Therefore, when dealing with series of floats, vectors, points or matrices, it is advisable to use kineticstoolkit.geometry's `matmul` instead of the @ operator, which will garranty to follow Kinetics Toolkit's array convention. For example among others:

**Matrix multiplication between a matrix and a series of points**

In [ ]:
# A single rigid transformation matrix that translates 3 units in x
T = np.array([[[1., 0., 0., 3.],
               [0., 1., 0., 0.],
               [0., 0., 1., 0.],
               [0., 0., 0., 1.]]])

# times a series of 5 points
points = np.array([[0., 1., 0., 1.],
                   [1., 1., 0., 1.],
                   [2., 1., 0., 1.],
                   [3., 1., 0., 1.],
                   [4., 1., 0., 1.]])

ktk.geometry.matmul(T, points)  # gives a series of 5 points

**Multiplication between a series of floats and a series of vectors**

In [ ]:
# A series of 5 floats
floats = np.array([0., 0.5, 1., 1., 1.])

# times a series of 5 vectors
vectors = np.array([[0., 1., 0., 0.],
                    [1., 1., 0., 0.],
                    [2., 1., 0., 0.],
                    [3., 1., 0., 0.],
                    [4., 1., 0., 0.]])

ktk.geometry.matmul(floats, vectors)  # gives a series of 5 vectors

Series of rotations
------------------------------------

The [geometry](api/kineticstoolkit.geometry.rst) module provides two functions for manipulating series of rotations:

- [create_rotation_matrices](api/kineticstoolkit.geometry.rst#kineticstoolkit.geometry.create_rotation_matrices): A wrapper to `scipy.transform.Rotation.from_euler` that creates Nx4x4 series rigid transformations based on rotation axes and angles.
- [get_euler_angles](api/kineticstoolkit.geometry.rst#kineticstoolkit.geometry.create_rotation_matrices): A wrapper to `scipy.transform.Rotation.as_euler` that extracts Euler angles from Nx4x4 series of rigid transformations.

Series of reference frames
-----------------------

![Right humerus reference frame](_static/geometry/humerus_reference_frame.png){ width=200px }


The [geometry](api/kineticstoolkit.geometry.rst) module provides the [create_reference_frames](api/kineticstoolkit.geometry.rst#kineticstoolkit.geometry.create_reference_frames) function to generate series of reference frames based on marker positions.

In this example, we will create the series of N rigid transformations that corresponds to the 3d position and orientation of the right humerus, based on three markers:

- GH: the Nx4 trajectory of the gleno-humeral joint;
- EL: the Nx4 trajectory of the lateral elbow epicondyle;
- EM: the Nx4 trajectory of the medial elbow epicondyle.
    
We will follow the recommendations of the International Society of Biomechanics [1]:

1. **The origin is GH**;

    ```origin = GH```

2. **The y axis is the line between GH and the midpoint of EL and EM, pointing to GH**;

    ```y = GH - (EL + EM) / 2```

3. **The x axis is the normal to the GH-EL-EM plane, pointing forward**;

    We define the yz plane (in yellow) by a second vector (in yellow) that is in this plane.
    The x axis will then be generated by the cross product of y and yz.
    
    ```yz = EL - EM```

4. **The z axis is perpendicular to x and y, pointing to the right**.

    ```reference_frames = ktk.geometry.create_reference_frames(origin=origin, y=y, yz=yz)```
    
    This yields a series of rigid transformation matrices that express the humerus reference frame shown in the figure above.

[1] G. Wu et al., "ISB recommendation on definitions of joint
       coordinate systems of various joints for the reporting of human joint
       motion - Part II: shoulder, elbow, wrist and hand," Journal of
       Biomechanics, vol. 38, no. 5, pp. 981--992, 2005.


Calculating local and global coordinates
---------------------
Finally, the [geometry](api/kineticstoolkit.geometry.rst) module provides function to change the reference frame used to express points, vectors and matrices from local to global coordinates and vice versa:

- [get_local_coordinates](api/kineticstoolkit.geometry.rst#kineticstoolkit.geometry.get_local_coordinates): Express global coordinates inside a local reference frame;
- [get_global_coordinates](api/kineticstoolkit.geometry.rst#kineticstoolkit.geometry.get_global_coordinates): Express local coordinates in global coordinates.

For more information on geometry, please check the [API Reference for the geometry module](api/kineticstoolkit.geometry.rst).